In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

print(tf.__version__)

%load_ext tensorboard

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

2022-12-06 13:28:57.843782: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 13:28:58.502571: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-06 13:28:58.834280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-06 13:28:58.834314: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

2.10.0


In [ ]:

def inputsoutputsAllfiles(trainIterations, filepatharray, Timestepsize, maxtimestep):

    def xyz(iteration, filepath, Timestepsize, Timestep):

        Time = Timestep*Timestepsize 
        if Time%1 == 0:
            Time =int(Time)
        else: 
            Time = round(Time,2)
        my_file = open(filepath + str(Time) +"/solidsCellD_iteration"+str(iteration))
        string_list = my_file.readlines()
        index=18
        string_list = string_list[index+2:len(string_list)] 
        string_list = string_list[0:len(string_list)-4] 
        i=0
        for i in range(len(string_list)):
            string_list[i]=string_list[i].replace("(","")
            string_list[i]=string_list[i].replace(")","")
            string_list[i]=string_list[i].split()
        arr=np.array(string_list)
        arr= arr.astype(np.float64)
        x = arr[:,0]
        y = arr[:, 1]
        z = arr[:, 2]

        return x, y, z #, arr
        
      
    def maxIteration (Timestep, Timestepsize, filepath):
        Time = Timestep*Timestepsize
        Time = round(Time,2)   # Rounds to deicimal place
        file = filepath + "residual.dat"

        df = pd.read_csv(file)

        df.columns = ['One'] #Labels first Column

        df['One'] = df['One'].astype('string') # Converts to string

        new = df['One'].str.split(' ', expand = True) #splits into new dataframe on spaces

        df['TimeStep']= new[0].astype('float')  # labels new columns and inserts in dataframe
        df['solverPerfInitRes']= new[1].astype('float')  # labels new columns and inserts in dataframe
        df['residualvf']= new[2].astype('float')  # labels new columns and inserts in dataframe
        df['materialResidual']= new[3].astype('float')  # labels new columns and inserts in dataframe
        df["IterationNumber"]= new[4].astype('float') # labels new columns and inserts in dataframe

        df.drop(columns =['One'], inplace = True) #Removes original column

        array = np.array
        array = []

        for i in range(len(df)):        #loops through residual file
            if df['TimeStep'][i] == Time:   # Finds values for a timestep
                x = df['IterationNumber'][i]
                array.append(x)            # Creates arrays of iteration numbers
                
        max = np.max(array) # returns max of array
        max=int(max)
        return max
    
    def inputsoutputsAllTimesteps(trainIterations, filepath, Timestepsize):

        def inputsoutputsTimestep(trainIterations, filepath, Timestepsize, Timestep):

            max = maxIteration(Timestep, Timestepsize, filepath)

            x,y,z = xyz(1, filepath, Timestepsize, Timestep)
            matrix = np.array(np.vstack((x,y,z))) #Gives matrix a starting value which is removed later

            for i in range(0,trainIterations): # loops for each iteration up to trainIteration
                x,y,z = xyz(i,filepath, Timestepsize, Timestep)
                arr = np.vstack((x,y,z))  # gives array with x, y, z values for each iteration
                matrix = np.concatenate((matrix, arr))   # combines arrays

            inputs = matrix[3:,:] # removes initial value

            xmax, ymax, zmax = xyz(max, filepath, Timestepsize, Timestep)
            outputs = np.vstack((xmax,ymax,zmax)) # array of values at max iteration

            inputs = np.transpose(inputs)  # multiple columns : (x1, y1, z1, x2, y2, z2 ...)
            outputs = np.transpose(outputs)

            return inputs, outputs

        arrinputs = np.ones((1, trainIterations*3)) # first row to be removed after
        arroutputs = np.ones((1, 3))

        #combines inputs and outputs of all timesteps

        for Timestep in range(1,maxtimestep+1):
            inputs, outputs = inputsoutputsTimestep(trainIterations, filepath, Timestepsize, Timestep)
            arrinputs = np.vstack((arrinputs,inputs))
            arroutputs = np.vstack((arroutputs,outputs))

        arrinputs = arrinputs[1:,:] #removes oriinal row of ones
        arroutputs = arroutputs[1:,:]#removes oriinal row of ones

        outputs = arroutputs
        inputs = arrinputs

        return inputs, outputs   # Returns the inputs and outputs of All timesteps for one file

    arrinputs = np.ones((1, trainIterations*3))
    arroutputs = np.ones((1, 3))

    for i in range(0,len(filepatharray),1):
        inputs, outputs = inputsoutputsAllTimesteps(trainIterations, filepatharray[i], Timestepsize)
        arrinputs = np.vstack((arrinputs,inputs))
        arroutputs = np.vstack((arroutputs,outputs))

    arrinputs = arrinputs[1:,:] #removes oriinal row of ones
    arroutputs = arroutputs[1:,:]#removes oriinal row of ones

    outputs = arroutputs
    inputs = arrinputs

    return inputs, outputs

In [ ]:
filepath1 = "../mesh10/plateHole/"

filepatharray = [filepath1]

In [ ]:
# trainIterations = input("Enter your training iteration value: ")
trainIterations = 20

Timestepsize = 1
maxtimestep = 1

inputs, outputs = inputsoutputsAllfiles(trainIterations, filepatharray, Timestepsize, maxtimestep)

In [ ]:
# define model

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

X = inputs
y = outputs

scalerI = StandardScaler().fit(X) #FitScaler
scalerO = StandardScaler().fit(y)


scaledX = scalerI.transform(X) # Scale X
scaledY = scalerO.transform(y)

In [ ]:
# Ask for number of epocs

# answer_epochs = input("Number of epochs?: ")
# answer_epochs = int(answer_epochs)

answer_epochs = 500

In [ ]:
# Ask if solidProperties is ready

# answer_solidsProp = input("Is solidProperties ready?: (y/n)")

answer_solidsProp = "y"

In [ ]:
# Ask to train or load model

# answer_trainOrLoadModel = input("Train or load model?: (t/l) ")

answer_trainOrLoadModel = "t"

In [ ]:
HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([100, 200, 300]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([100, 150, 200]))
HP_NUM_UNITS_3 = hp.HParam('num_units_3', hp.Discrete([25, 50, 75]))

HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS_1, HP_NUM_UNITS_2, HP_NUM_UNITS_3, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )


In [ ]:
def train_test_model(hparams):

  model = Sequential()
  model.add(Dense(hparams[HP_NUM_UNITS_1], input_dim=len(X[0,:]), 
    activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(hparams[HP_NUM_UNITS_2], 
    activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(hparams[HP_NUM_UNITS_3], 
    activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(3))

  # model = tf.keras.models.Sequential([
  #   tf.keras.layers.Flatten(),
  #   tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
  #   tf.keras.layers.Dense(3),
  # ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='mean_squared_error',
      metrics=['accuracy'],
  )

  log_dir = "logs/fit/" 
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


  history = model.fit(scaledX, scaledY, epochs=answer_epochs, verbose=1)
  accuracy = model.evaluate(scaledX, scaledY, verbose=0)
  return accuracy

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    # tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [ ]:
!rm -r logs

session_num = 0

for num_units_1 in HP_NUM_UNITS_1.domain.values:
    for num_units_2 in HP_NUM_UNITS_2.domain.values:
        for num_units_3 in HP_NUM_UNITS_3.domain.values:
                for optimizer in HP_OPTIMIZER.domain.values:
                    hparams = {
                        HP_NUM_UNITS_1: num_units_1,
                        HP_NUM_UNITS_2: num_units_2,
                        HP_NUM_UNITS_3: num_units_3,
                        HP_OPTIMIZER: optimizer,
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    run('logs/hparam_tuning/' + run_name, hparams)
                    session_num += 1

--- Starting trial: run-0
{'num_units_1': 50, 'num_units_2': 10, 'optimizer': 'adam'}
Epoch 1/1000
313/313 [==============================] - 0s 604us/step - loss: 0.0538 - accuracy: 0.9406
Epoch 2/1000
313/313 [==============================] - 0s 582us/step - loss: 5.4588e-04 - accuracy: 0.9882
Epoch 3/1000
313/313 [==============================] - 0s 611us/step - loss: 4.1438e-04 - accuracy: 0.9895
Epoch 4/1000
313/313 [==============================] - 0s 595us/step - loss: 3.5562e-04 - accuracy: 0.9903
Epoch 5/1000
313/313 [==============================] - 0s 590us/step - loss: 3.7187e-04 - accuracy: 0.9910
Epoch 6/1000
313/313 [==============================] - 0s 625us/step - loss: 3.2788e-04 - accuracy: 0.9913
Epoch 7/1000
313/313 [==============================] - 0s 567us/step - loss: 2.5872e-04 - accuracy: 0.9915
Epoch 8/1000
313/313 [==============================] - 0s 531us/step - loss: 2.3119e-04 - accuracy: 0.9919
Epoch 9/1000
313/313 [==============================] 

KeyboardInterrupt: 